In [1]:
# Extract SO-CI information from MOLPRO outputs for atoms
#   Read exptl data from Excel file, combine with weights to get E_so
#   The experimental Excel file is generated by get_NIST_atomic_data.ipynb
# More robust J assignments (attempted)
# KKI 8/30/2024
# Working for difficult Ta case (9/25/2024)
#    (1) get it working for even-electron case (Kr test)
#    (2) transparent handling of missing exptl levels in eq. (2)
import re, sys, os
os.environ['OMP_NUM_THREADS'] = '1'  # to avoid possible memory leak in Kmeans
import numpy as np
import pandas as pd
from collections import Counter
#import random
#import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import chem_subs as chem
#import molpro_subs as mpr
import molpro_subs2 as m2

pd.set_option('display.max_rows', None)
np.set_printoptions(suppress=True)

In [2]:
# how to format DataFrames
fmt = {'Eshift': '{:.1f}', 'degen': '{:d}'}
for col in ['J', 'Ecalc', 'E_dif', 'Erel', 'Eshift', 'err', 'Eterm', 'cm-1', 'S',
           'wmean', 'wstds', 'uwmean', 'uwstds', 'change', 'rwmse', 'Erel_spread',
           'Edav_range_cm']:
    fmt[col] =  fmt['Eshift']
for col in ['dif', 'Theory', 'ecm', 'SOC', 'RMSE']:
    fmt[col] = '{:.2f}'
fmt['weight'] = '{:.6f}'

### Specify Molpro SO-CI output file

In [3]:
#fname = 'Mo_S1Q18_acvqz-pp.out'  # degenerate levels are problem
fname = 'pb_3PDPS1DSPDS_actzpp.pro'
#fname = 'Tc_6SQ26_acvt-pp.out'

In [4]:
# my atom subdirectory names look like "Ar_I" (for neutral argon)
el = fname.split('_')[0].capitalize()
fdir = r'C:\Users\irikura\OneDrive - NIST\Karl\atomic_SOC\calculations\{:s}_I'.format(el)
#fdir = r'C:\Users\dagbaglo\Desktop\So-ci_energy\{:s}_I'.format(el)

In [5]:
fsoc = os.sep.join([fdir, fname])
print(f'Reading MOLPRO file')
print(fsoc)

Reading MOLPRO file
C:\Users\irikura\OneDrive - NIST\Karl\atomic_SOC\calculations\Pb_I\pb_3PDPS1DSPDS_actzpp.pro


In [6]:
# Identify the different sections of the output file
major_sections, linenos = m2.identify_sections(fsoc)
tcpu, treal, tstr = m2.final_times(fsoc, verbose=False)
print(f'Calculation time = {treal/3600:.1f} hours')
if True:
    print('Major sections:')
    for k, v in major_sections.items():
        print(f'   {k:<11s}   {len(v)} text blocks')

Calculation time = 20.3 hours
Major sections:
   input         1 text blocks
   header        1 text blocks
   integrals     1 text blocks
   rhf           1 text blocks
   multi         1 text blocks
   mrci          2 text blocks
   soci          1 text blocks


In [7]:
# section "header"
basisset = m2.basisset_name(major_sections['header'][-1])
# section "integrals"
PG = m2.point_group(major_sections['integrals'][-1])
print(f'Computational point group = {PG}')
if PG != 'Ci':
    chem.print_err('', 'Ci point group is required for this analysis')
nprim = m2.nbf_primitive(major_sections['integrals'][-1])
nbf = m2.nbf(major_sections['integrals'][-1])
print(f'{basisset} basis set')
print(f'    {nprim} primitives')
print(f'    {nbf} contracted basis functions')
crd = m2.coordinates(major_sections['integrals'][-1])
atom = crd[-1]['el']
if atom != el:
    chem.print_err('', f'This looks like the wrong atom ({atom}) for the filename ({el})')
Qtot = m2.nuclear_charge_total(major_sections['integrals'][-1])
print(f'Atom "{atom}" with nuclear charge = {Qtot}')
Zel = chem.elz(atom, 'Z')
if Zel > Qtot:
    print(f'    pseudopotential replaces {Zel - Qtot} core electrons')

Computational point group = Ci
AUG-CC-PWCVTZ-PP basis set
    178 primitives
    96 contracted basis functions
Atom "Pb" with nuclear charge = 22
    pseudopotential replaces 60 core electrons


In [8]:
# section "rhf"
try:
    occup_hf = m2.hf_occup(major_sections['rhf'][-1])
    print('HF occupations: ', occup_hf)
    hf_results = m2.scf_result('RHF', major_sections['rhf'][-1])
    print('HF energy = {:.6f} for state {:s}'.format(hf_results['E'], hf_results['Label']))
    orbtitle, dfHForb = m2.parse_orbitals(major_sections['rhf'][-1])
    nel_HF = 0
    for otype, occs in occup_hf.items():
        if otype == 'alpha/beta':
            nel_HF += 2 * sum(occs)
        else:
            nel_HF += 1 * sum(occs)
    print(f'HF has {nel_HF} electrons (charge = {Qtot - nel_HF})')
    print(orbtitle)
    m2.color_by_orb(dfHForb)
except KeyError:
    print('No Hartree-Fock step found')
    nel_HF = 0

HF occupations:  {'alpha': [7, 6], 'beta ': [7, 3]}
HF energy = -191.864227 for state 1.2
HF has 23 electrons (charge = -1)
ELECTRON ORBITALS


Orbital,Occ,Energy,Cen,Mu,Typ,Coeff
1.1,2.000000,-5.949070,1,1,s,1.00046
2.1,2.000000,-0.849240,1,1,d0,0.99980
3.1,2.000000,-0.849240,1,1,d1+,0.51667
3.1,2.000000,-0.849240,1,1,d1-,0.85562
4.1,2.000000,-0.849240,1,1,d1+,0.85515
4.1,2.000000,-0.849240,1,1,d1-,-0.51668
5.1,2.000000,-0.849240,1,1,d2-,0.98087
6.1,2.000000,-0.849240,1,1,d2+,0.98134
7.1,2.000000,-0.352450,1,2,s,0.96592
1.2,2.000000,-3.631530,1,1,pz,0.99314


In [9]:
# break section "multi" into sub-sections
multisec = m2.multi_sections(major_sections['multi'][-1])
multisec.keys()

dict_keys(['top', 'iterations', 'results', 'trans', 'natorb', 'civector', 'tail'])

In [10]:
# parsing MULTI sub-sections
dynfac = m2.get_dynfac(multisec['top'][-1])
orbspace = m2.orbital_spaces(multisec['top'][-1])
statesym = m2.state_symmetry_groups(multisec['top'][-1])
convergence = m2.multi_convergence(multisec['iterations'][-1])
weights = m2.multi_weights(multisec['iterations'][-1])
dfiter = m2.multi_iterations(multisec['iterations'][-1])
dfstates = m2.multi_results(multisec['results'])
dfexpec = m2.multi_expec(multisec['trans'][-1])
dftrans = m2.multi_transmom(multisec['trans'][-1])
orbtitle, dfNO = m2.parse_orbitals(multisec['natorb'][-1])
ddfcivec, dEcas = m2.multi_civecs(multisec['civector'])

In [11]:
nel_CAS = nactel = statesym[0]['nelec']
nactorb = sum(orbspace['active'])
print(f'CASSCF active space is ({nactel}/{nactorb}) with active orbitals {orbspace["active"]}')
if 'closed-shell' in orbspace.keys():
    print(f'    closed orbitals are {orbspace["closed-shell"]}')
    nel_CAS += 2 * sum(orbspace["closed-shell"])
else:
    print( '    There are no "closed" orbitals')
if 'frozen' in orbspace.keys():
    print(f'    frozen orbitals are {orbspace["frozen"]}')
    nel_CAS += 2 * sum(orbspace["frozen"])
else:
    print( '    There are no "frozen" orbitals')
print(f'    charge = {Qtot - nel_CAS}')
# Count the states
mult_count = {}
ncas = 0
for st in statesym:
    mult = st['spin']
    mult_count[mult] = st['nstates'] + mult_count.get(mult, 0)
    ncas += st['nstates']
print(f'{ncas} CASSCF states:')
for mult, n in mult_count.items():
    print(f'   {n:3d} {mult}')
    
# Show the state weights, renormalized for reading convenience
print('CASSCF relative state weights (subject to rounding error):')
uweights = m2.unnormalize_cas_weights(weights)
for k, wts in uweights.items():
    print('    ', np.round(wts, 1))
    if max(wts) > wts[0] + 0.01:
        print('    *** putting a heavy weight on and excited can cause trouble ***')
    
# Are <L**2> values clean?
ilsq = np.rint(dfexpec['L**2'])
maxdev = np.abs(ilsq - dfexpec['L**2']).max()
if maxdev:
    print(f'Largest deviation of <L**2> from integer = {maxdev:.1e}')
else:
    print('Values of <L**2> are clean')
if nel_CAS == nel_HF:
    CAS_rel_HF = dfstates.E.min() - hf_results['E']
    print(f'For the ground state, [E(CASSCF) - E(HF)] = {CAS_rel_HF:.6f}')
    if CAS_rel_HF >= 0:
        print('   *** this difference is usually negative')
        print('   *** consider using a heavier CASSCF weight on the ground term')
elif nel_HF > 0:
    print(f'CASSCF and HF have different numbers of electrons ({nel_CAS} and {nel_HF})')
print()
print(orbtitle)
orb_styler = m2.color_by_orb(dfNO)

CASSCF active space is (4/7) with active orbitals [1, 6]
    closed orbitals are [6, 3]
    There are no "frozen" orbitals
    charge = 0
27 CASSCF states:
    12 Triplet
    15 Singlet
CASSCF relative state weights (subject to rounding error):
     [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
     [1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Values of <L**2> are clean
CASSCF and HF have different numbers of electrons (22 and 23)

NATURAL ORBITALS (STATE AVERAGED)


Orbital,Occ,Energy,Cen,Mu,Typ,Coeff
1.1,2.000000,-6.272660,1,1,s,0.99928
2.1,2.000000,-1.173120,1,1,d2-,1.00073
3.1,2.000000,-1.173120,1,1,d2+,1.00073
4.1,2.000000,-1.173120,1,1,d1-,1.00073
5.1,2.000000,-1.173120,1,1,d1+,1.00073
6.1,2.000000,-1.173120,1,1,d0,1.00073
7.1,1.971630,-0.640970,1,2,s,1.02360
1.2,2.000000,-3.955360,1,1,py,0.99946
2.2,2.000000,-3.955360,1,1,px,0.99946
3.2,2.000000,-3.955360,1,1,pz,0.99946


In [12]:
if False:
    # print results from parsing MULTI output
    print(f'DYNW = {dynfac}')
    print('Spaces: ', orbspace)
    print('CASSCF state groups:')
    for g in statesym:
        print('   ', g)
    print(convergence)
    print('CASSCF state weights:')
    for k, v in weights.items():
        print(f'  {k:>2s}: ', v)
    display(dfiter)
    display(dfstates)
    display(dfexpec)
    for op, df in dftrans.items():
        print(f'Operator {op}')
        display(df)
    print(orbtitle)
    display(dfNO)
    for k, df in ddfcivec.items():
        print(k, dEcas[k])
        display(df.head())

In [13]:
ineg = np.argwhere(dfexpec['L**2'] < 0)
if len(ineg) > 0:
    print(f'*** Negative values of <L**2> as low as {dfexpec["L**2"].min()} ***')
    display(dfexpec[dfexpec['L**2'] < 0])
    ismall = np.argwhere(dfexpec['L**2'].abs() < 1.e-5).flatten().tolist()
    #print(ismall)
    if len(ismall):
        print('Setting small values to zero')
        dfexpec.loc[ismall, 'L**2'] = 0

In [14]:
ineg = np.argwhere(dfexpec['LZLZ'] < 0)
if len(ineg) > 0:
    print(f'*** Negative values of <LZLZ> as low as {dfexpec["LZLZ"].min()} ***')
    display(dfexpec[dfexpec['LZLZ'] < 0])
    ismall = np.argwhere(dfexpec['LZLZ'].abs() < 1.e-5).flatten().tolist()
    #print(ismall)
    if len(ismall):
        print('Setting small values to zero')
        dfexpec.loc[ismall, 'LZLZ'] = 0

In [15]:
# Summarize CASSCF results
dfcas = dfstates[['Label', 'irrep', 'E']].copy()
Svals = []
for g in statesym:
    for i in range(g['nstates']):
        Svals.append(chem.MULTSPIN[g['spin']])
dfcas.insert(2, 'S', Svals)
dfcas['L**2'] = dfexpec['L**2']
dfcas['LZ'] = np.sqrt(dfexpec['LZLZ'])
dfcas['L'] = np.sqrt(dfexpec['L**2']).astype(int)
tsymb = []
for S, L, irr in zip(dfcas.S, dfcas.L, dfcas.irrep):
    parity = 3 - 2*irr
    trm = chem.term_symbol(L, S, parity, linear=False)
    tsymb.append(trm)
dfcas['term'] = tsymb
print('CASSCF states')
dfcas

CASSCF states


,Label,irrep,S,E,L**2,LZ,L,term
0,1.1 Triplet,1,1,-191.835161,2.0,1.000000,1,3P
1,2.1 Triplet,1,1,-191.835161,2.0,0.000036,1,3P
2,3.1 Triplet,1,1,-191.835161,2.0,1.000000,1,3P
3,4.1 Triplet,1,1,-191.665228,6.0,1.972232,2,3D
4,5.1 Triplet,1,1,-191.665228,6.0,2.000000,2,3D
5,6.1 Triplet,1,1,-191.665228,6.0,1.000000,2,3D
6,7.1 Triplet,1,1,-191.665228,6.0,1.000000,2,3D
7,8.1 Triplet,1,1,-191.665228,6.0,0.332124,2,3D
8,9.1 Triplet,1,1,-191.659658,2.0,1.000000,1,3P
9,10.1 Triplet,1,1,-191.659658,2.0,1.000000,1,3P


In [16]:
dfcasterm = m2.collect_atomic_terms(dfcas, 'E')
nterm = len(dfcasterm)
print(f'There are {ncas} CASSCF states in {nterm} terms')
# Add J values
Jvals = [chem.possible_J_from_term(trm) for trm in dfcasterm['term']]
dfcasterm['J_vals'] = Jvals
display(dfcasterm)

There are 27 CASSCF states in 9 terms


,term,E,Erel,E_range_cm,irrep,L,S,idx,Labels,J_vals
0,3P,-191.835161,0.0,0.0,1,1,1.0,"[0, 1, 2]","[1.1, 2.1, 3.1]","[0.0, 1.0, 2.0]"
1,1D,-191.801237,7445.3,0.0,1,2,0.0,"[12, 13, 14, 15, 16]","[1.1, 2.1, 3.1, 4.1, 5.1]",[2.0]
2,1S,-191.767765,14791.7,0.0,1,0,0.0,[17],[6.1],[0.0]
3,1P,-191.670332,36175.7,0.0,1,1,0.0,"[18, 19, 20]","[7.1, 8.1, 9.1]",[1.0]
4,3D,-191.665228,37295.9,0.0,1,2,1.0,"[3, 4, 5, 6, 7]","[4.1, 5.1, 6.1, 7.1, 8.1]","[1.0, 2.0, 3.0]"
5,3P,-191.659658,38518.4,0.0,1,1,1.0,"[8, 9, 10]","[9.1, 10.1, 11.1]","[0.0, 1.0, 2.0]"
6,3S,-191.657577,38975.1,0.0,1,0,1.0,[11],[12.1],[1.0]
7,1D,-191.648109,41053.1,0.0,1,2,0.0,"[21, 22, 23, 24, 25]","[10.1, 11.1, 12.1, 13.1, 14.1]",[2.0]
8,1S,-191.632381,44505.0,0.0,1,0,0.0,[26],[15.1],[0.0]


In [17]:
# Parse MRCI results and summarize in DataFrame
dfmrci = pd.DataFrame()
for imrci, sec in enumerate(major_sections['mrci']):
    print(f'MRCI calculation #{imrci+1}')
    mrcisec = m2.mrci_sections(sec)
    mrci_meta = m2.mrci_info(mrcisec['top'][0])
    mrci_iter = m2.mrci_iterations(mrcisec['iterations'][0])
    mrci_results = m2.mrci_results(mrcisec['results'][0])
    nstate = len(mrci_results['state'])
    print(f'    {mrci_meta["smult"]}, irrep {mrci_meta["irrep"]}')
    print(f'    {nstate} states')
    # Report on orbital spaces in the MRCI
    print('    orbital spaces, by irrep')
    for sp in ['core', 'closed', 'active', 'external']:
        print('\t{:10s} {}'.format(sp, mrci_meta['spaces'].get(sp, [])))
    lbll =  []  # list of state labels
    c0rot = []  # list of C0 (rotated) values
    El =    []  # list of energies
    davl =  []  # list of Davidson-corrected energies (rotated ref)
    erefl = []  # list of reference energies
    spinmult = mrci_meta['smult']
    S = chem.MULTSPIN[spinmult]
    irrep = mrci_meta['irrep']
    for lbl, v in mrci_results['state'].items():
        lbll.append(lbl)
        try:
            c0rot.append(v['C0']['rotated'])
            davl.append(v['Energy']['davidson']['rotated'])
        except KeyError:
            # no "rotated" values if there is only one state
            c0rot.append(v['C0']['relaxed'])
            davl.append(v['Energy']['davidson']['relaxed'])
        El.append(v['Energy']['total'])
        erefl.append(v['Energy']['ref E'])

    # Get CASSCF (fixed) term composition of each MRCI state
    fixref = m2.coefficients_of_refs(mrcisec['results'][0])  # coeffs of CASSCF refs
    subcas = dfcas[dfcas.S == S]
    if nstate == 1:
        # 'fixref' is [0] because that text block is not available
        # just assign 100% to the only CASSCF reference state
        ilead = np.array([0])
        fixref = np.array([])
        casterm = subcas.iloc[0]['term']
        cascomp = [{casterm: 1.}]
    else:
        ilead = np.zeros(fixref.shape[1]).astype(int)
        cascomp = []
    for ist in range(fixref.shape[0]):
        icas = np.argmax(np.abs(fixref[ist, :]))
        ilead[ist] = icas
        termd = {}
        for icas, c in enumerate(fixref[ist, :]):
            casterm = subcas.iloc[icas]['term']
            termd[casterm] = termd.get(casterm, 0) + c*c
        cascomp.append(termd)
    if len(set(ilead)) < len(ilead):
        print('    ** Warning: one CASSCF state leads more than one MRCI state')
    reflbl = [subcas.iloc[i]['Label'] for i in ilead]
    lz = [subcas.iloc[i]['LZ'] for i in ilead]
    x = [v for v in mrci_iter['init_ref'].values()][:nstate]
    init_refE = [x[i] for i in ilead]
    terml = [subcas.iloc[i]['term'] for i in ilead]
    data = {'Label': lbll, 'irrep': irrep, 'S': S, 'E': El, 'Edav': davl,
            'C0': c0rot, 'Eref': erefl, 'init_ref': init_refE, 'iref_nr': ilead + 1,
            'irlbl': reflbl, 'term': terml, 'LZ': lz, 'CASterm': cascomp}
    dfci = pd.DataFrame(data)
    dfmrci = pd.concat([dfmrci, dfci], ignore_index=True)
# add column for difference between reference energy and corresponding CASSCF
d_ref = dfmrci.Eref - dfmrci.init_ref
dfmrci.insert(8, 'D_ref', d_ref)

MRCI calculation #1
    Triplet, irrep 1
    12 states
    orbital spaces, by irrep
	core       []
	closed     []
	active     [1, 6]
	external   [40, 40]
MRCI calculation #2
    Singlet, irrep 1
    15 states
    orbital spaces, by irrep
	core       []
	closed     []
	active     [1, 6]
	external   [40, 40]


In [18]:
# Check for term contamination of MRCI states
maxcontam = 0  # largest term contamination across all MRCI states
for ici, row in dfmrci.iterrows():
    tlead = row.term
    twt = row.CASterm
    tmax = max(twt, key=twt.get)
    if tlead != tmax:
        print(f'*** Term confusion for MRCI state! ***')
        display(row.to_frame().T)
    dcontam = {}
    del twt[tlead]  # what is left is contamination
    for k in list(twt.keys()):
        if twt[k] == 0:
            del twt[k]  # omit zero contaminants
    maxcontam = max([maxcontam] + list(twt.values()))
print(f'Largest MRCI term contamination = {maxcontam:.1e}')
# Rename CASterm to CAScontam
dfmrci.rename(columns={'CASterm': 'CAScontam'}, inplace=True)

Largest MRCI term contamination = 2.0e-10


In [19]:
nmrci = len(dfmrci)
#dfmrci
dfciterm = m2.collect_atomic_terms(dfmrci, 'Edav')
print(f'There are {nmrci} MRCI states in {len(dfciterm)} terms')
if nterm != len(dfciterm):
    chem.print_err('', 'Different number of terms from CASSCF and from MRCI')
# Make prefixes enumerative
dfciterm['term'] = chem.enumerative_prefix(dfciterm.term.values, always=False)
termsIn = set(dfciterm.term)
print('MRCI terms:')
styler = dfciterm.style
styler = styler.apply(lambda x: ["background: yellow" if abs(v) > 5 else "" for v in x], 
              subset=pd.IndexSlice[['Edav_range_cm']])
styler.format(fmt)

There are 27 MRCI states in 9 terms
MRCI terms:


,term,Edav,Erel,Edav_range_cm,irrep,L,S,idx,Labels
0,(1)3P,-192.502341,0.0,0.0,1,1,1.0,"[0, 1, 2]","['1.1', '2.1', '3.1']"
1,(1)1D,-192.477229,5511.6,0.0,1,2,0.0,"[12, 13, 14, 15, 16]","['1.1', '2.1', '3.1', '4.1', '5.1']"
2,(1)1S,-192.437913,14140.4,0.0,1,0,0.0,[17],['6.1']
3,1P,-192.319995,40020.4,0.0,1,1,0.0,"[18, 19, 20]","['7.1', '8.1', '9.1']"
4,3D,-192.315212,41070.1,0.1,1,2,1.0,"[3, 4, 5, 6, 7]","['4.1', '5.1', '6.1', '7.1', '8.1']"
5,(2)3P,-192.311117,41969.0,0.1,1,1,1.0,"[8, 9, 10]","['9.1', '10.1', '11.1']"
6,3S,-192.308221,42604.6,0.0,1,0,1.0,[11],['12.1']
7,(2)1D,-192.303412,43660.0,0.4,1,2,0.0,"[21, 22, 23, 24, 25]","['10.1', '11.1', '12.1', '13.1', '14.1']"
8,(2)1S,-192.290324,46532.5,0.0,1,0,0.0,[26],['15.1']


In [20]:
# section "SOintegrals"
if 'SOintegrals' in major_sections.keys():
    SOintgrl = m2.SO_integrals(major_sections['SOintegrals'][0])
    #print(SOintgrl)

In [21]:
# break section "soci" into sub-sections
sosec = m2.soci_sections(major_sections['soci'][0])
sosec.keys()

dict_keys(['matel_comput', 'so_calc', 'so_levels', 'so_vectors', 'so_compos', 'tail'])

In [22]:
# SOCI sub-section 'matel_comput'
hlsdiag = m2.soci_replacements(sosec['matel_comput'][0])
n_cistates = sum([x['nstate'] for x in hlsdiag.values()])
print(f'There are {n_cistates} states in the HLSDIAG list:')
# Present a table of old and new diagonal energies, as a check
data = {'sym': [], 'S': [], 'old': [], 'HLSDIAG': []}
for rec, calc in hlsdiag.items():
    for old, new in zip(calc['old'], calc['new']):
        data['sym'].append(calc['sym'])
        data['S'].append(calc['S'])
        data['old'].append(old)
        data['HLSDIAG'].append(new)
    dfhlsdiag = pd.DataFrame(data=data)
    dfhlsdiag['diff'] = dfhlsdiag.HLSDIAG - dfhlsdiag.old
styler = dfhlsdiag.style
warnThresh = 0.5  # (Hartree) highlight changes in diagonal larger than this
styler = styler.apply(lambda x: ["background: yellow" if (abs(v) > warnThresh) or (v > 0) else "" for v in x], 
              subset=pd.IndexSlice[['diff']])
display(styler.format(fmt))
# check for zeros
if (np.array(data['HLSDIAG']) == 0).any():
    chem.print_err('', 'Zero values in HLSDIAG')
mat_elems = m2.soci_matelems(sosec['matel_comput'][0])
if mat_elems:
    print(mat_elems)

There are 27 states in the HLSDIAG list:


,sym,S,old,HLSDIAG,diff
0,1,1.0,-192.448375,-192.502341,-0.053966
1,1,1.0,-192.448375,-192.502341,-0.053966
2,1,1.0,-192.448375,-192.502341,-0.053966
3,1,1.0,-192.264233,-192.315213,-0.050980
4,1,1.0,-192.264233,-192.315212,-0.050979
5,1,1.0,-192.264233,-192.315212,-0.050979
6,1,1.0,-192.264233,-192.315212,-0.050979
7,1,1.0,-192.264233,-192.315212,-0.050979
8,1,1.0,-192.260112,-192.311117,-0.051005
9,1,1.0,-192.260112,-192.311117,-0.051005


In [23]:
# SOCI sub-section 'basis_prop'
if 'basis_prop' in sosec.keys():
    basprop = m2.soci_basis_prop(sosec['basis_prop'][0], n_cistates)
    print(basprop['DMZ'].diagonal())

In [24]:
# SOCI sub-section 'so_calc'
E0 = m2.soci_E0(sosec['so_calc'][0])
print(f'E0 = {E0:.6f} in the SO-CI')
somat = m2.soci_matrix(sosec['so_calc'][0])
dimen = somat['matrix'].shape[0]
print(f'There are {dimen} SO-CI states')

E0 = -192.502341 in the SO-CI
There are 51 SO-CI states


In [25]:
# Check for zero spin-orbit coupling
offdiag =somat['matrix'].copy()
np.fill_diagonal(offdiag, 0)
amax = np.max(np.abs(offdiag))
if amax == 0:
    print('*** Off-diagonal elements of spin-orbit matrix are all zero ***')
    print('There is no spin-orbit coupling')
    sys.exit(1)

In [26]:
# Add to basis-state info: MRCI and term parentage, LZ etc.
for i, bas in enumerate(somat['basis']):
    S = bas['S']
    lbl = bas['State']
    subdf = dfmrci[(dfmrci.Label == lbl) & (dfmrci.S == S)]
    #display(subdf)
    ici = subdf.index[0]
    bas['ici'] = ici
    for iterm, trow in dfciterm.iterrows():
        if ici in trow.idx:
            bas['iterm'] = iterm
            break
    # Add LZ
    bas['LZ'] = np.round(subdf.loc[ici, 'LZ'], 5)
    bas['JZ'] = set([abs(bas['Sz'] - bas['LZ']), abs(bas['Sz'] + bas['LZ'])])
    # find the minimum Jz
    minJ = min(bas['JZ'])
    # round minJ up
    minJ = np.rint(minJ - bas['Sz']) + bas['Sz']
    tlbl = dfciterm.loc[bas['iterm'], 'term']  # term label
    jposs = np.array(chem.possible_J_from_term(tlbl))
    bas['Jterm'] = set(jposs)
    # J cannot be less than Jz
    jposs = jposs[jposs >= minJ]
    bas['Jposs'] = set(jposs)

In [27]:
# SOCI sub-section 'so_levels'
so_energies = m2.soci_energies(sosec['so_levels'][0])
df_soE = pd.DataFrame(so_energies)
print(f'There are {len(df_soE)} spin-orbit levels')
df_soE

There are 51 spin-orbit levels


,E,Eshift,Erel
0,-192.551932,-10883.90,0.00
1,-192.519027,-3662.09,7221.81
2,-192.519027,-3662.09,7221.81
3,-192.519027,-3662.09,7221.81
4,-192.505430,-677.82,10206.08
5,-192.505430,-677.82,10206.08
6,-192.505429,-677.63,10206.27
7,-192.505429,-677.63,10206.27
8,-192.505429,-677.63,10206.27
9,-192.459938,9306.52,20190.42


In [28]:
SOCraw = min(so_energies['Eshift'])
print(f'From lowest level and lowest uncoupled term energy, raw theoretical SOCraw = {SOCraw:.2f} cm-1')

From lowest level and lowest uncoupled term energy, raw theoretical SOCraw = -10883.90 cm-1


In [29]:
# SO-CI sub-section 'so_vectors'
# In case of symmetry blocking, the last one should be the summary
so_vecs = m2.soci_vectors(sosec['so_vectors'][-1])
so_vecs.keys()

dict_keys(['basis', 'matrix'])

In [30]:
# check eigenvectors for normality
#    eigenvectors are columns of so_vecs['matrix']
tol = 1.e-7
mat = so_vecs['matrix']
for i in range(dimen):
    prod = np.dot(np.conjugate(mat[:, i]), mat[:, i])
    if np.abs(1 - prod) > tol:
        print(i, i, ':  ', prod)

In [31]:
# check eigenvectors for orthogonality
#    eigenvectors are columns of so_vecs['matrix']
mat = so_vecs['matrix']
for i in range(dimen):
    for j in range(i):
        prod = np.dot(np.conjugate(mat[:, i]), mat[:, j])
        if np.abs(prod) > tol:
            print(i, j, ':  ', np.abs(prod))

In [32]:
# SO-CI sub-section 'so_compos'
so_compos = m2.soci_composition(sosec['so_compos'][0])
so_compos.keys()

dict_keys(['basis', 'matrix'])

In [33]:
# check that all listings of basis states are consistent
for a, b, c in zip(somat['basis'], so_vecs['basis'], so_compos['basis']):
    for k in a.keys():
        if (k in b.keys()) and (k in c.keys()):
            if (a[k] != b[k]) or (a[k] != c[k]):
                print(a)
                print(b)
                print(c)
                print('----------------')

In [34]:
# check that composition is consistent with eigenvectors
magnit = np.conjugate(so_vecs['matrix']) * so_vecs['matrix']
# get differences in percent (printed by Molpro to 0.01% precision)
difmat = (magnit * 100) - so_compos['matrix']
dmax = np.abs(difmat).max()
print(f'Largest inconsistency between composition and eigenvectors = {dmax:.2f} %')

Largest inconsistency between composition and eigenvectors = 0.00 %


In [35]:
# Convert basis-state compositions (percent) to term compositions
use_printed = False  # use composition % as printed by Molpro (less precise)
if use_printed:
    # compositions are printed to 0.001% precision
    print('Using rounded compositions as printed by Molpro')
    magpct = so_compos['matrix']
else:
    # eigenvectors are orthonormal and printed to 1e-8 precision
    print('Using compositions derived from eigenvectors')
    magpct = np.real(magnit * 100)
term_compos = np.zeros((nterm, dimen))
for ibas in range(dimen):
    iterm = somat['basis'][ibas]['iterm']
    term_compos[iterm,:] += magpct[ibas,:]

Using compositions derived from eigenvectors


In [36]:
if False:
    # Display composition of one SO-CI state
    iso = 26
    wts = magpct[:, iso]  # basis-state weights
    dflevcomp = pd.DataFrame({'wt_bas': np.round(wts, 6)})
    dflevcomp['bas_Jposs'] = [x['Jposs'] for x in somat['basis']]
    dflevcomp['S'] = [x['S'] for x in somat['basis']]
    dflevcomp['Sz'] = [x['Sz'] for x in somat['basis']]
    dflevcomp['ici'] = [x['ici'] for x in somat['basis']]
    dflevcomp['iterm'] = [x['iterm'] for x in somat['basis']]
    dflevcomp['term'] = [dfmrci.loc[x['ici'], 'term'] for x in somat['basis']]
    dflevcomp['term_Jposs'] = [chem.possible_J_from_term(trm) for trm in dflevcomp['term']]
    display(dflevcomp.sort_values('wt_bas', ascending=False))

In [37]:
# Add J values to dfciterm
jpossl = []
for term in dfciterm.term:
    jposs = chem.possible_J_from_term(term)
    jpossl.append(jposs)
dfciterm['J'] = jpossl

In [38]:
def term_index(x):
    # Given a term symbol, return its index in dfciterm
    # Given an index, return the term symbol from dfciterm
    global dfciterm
    try:
        trm = dfciterm.at[x, 'term']
        return trm
    except:
        try:
            i = dfciterm.index[dfciterm.term == x].tolist()[0]
            return i
        except IndexError:
            # invalid term
            return None

In [39]:
# Get target J counts corresponding to the CASSCF terms
allJ = []
for jl in dfcasterm['J_vals']:
    allJ.extend(jl)
J_all = {float(k): v for k, v in Counter(allJ).items()}  # convert to plain float
print('Required level counts     :', J_all)
nlevels = len(allJ)
print(f'    There are {nlevels} J-levels')
Jxg = {k: int(v * (2*k+1)) for k, v in J_all.items()}
J_left = Jxg.copy()  # copy to be decremented
print('Required sublevel counts:', Jxg)
df_soE['J'] = None

Required level counts     : {0.0: 4, 1.0: 5, 2.0: 5, 3.0: 1}
    There are 15 J-levels
Required sublevel counts: {0.0: 4, 1.0: 15, 2.0: 25, 3.0: 7}


### Assign values of <em>J</em>  to levels

In [40]:
def find_possible_J(df_soE, basis_descr, magpct, style='term', thrpct=10.):
    '''
    Add set of possible J values to each level/row of df_soE
        also add their count
    Return nothing
    Args: df_soE       DataFrame of SO-CI levels
          basis_descr  list of dict; info about each basis state
          magpct       square array, pct weights of basis states, indices [ibas, iso]
          style        either 'term' or 'basis state'; which to use in determining J poss
          thrpct       pct threshold for including terms or basis states
    '''
    if style == 'term':
        choice = 'Jterm'  # key in basis-state description
    elif style == 'basis state':
        choice = 'Jposs'
    else:
        chem.print_err('', '*** Argument "style" must be either "term" or "basis state"')
    Jpossl = []  # list of sets
    for iso, row in df_soE.iterrows():
        bascomp = magpct[:, iso]  # composition vector for this level
        idx = np.argwhere(bascomp > thrpct).flatten()
        if len(idx) == 0:
            # in rare case that no basis state contributes 'thrpct' %
            idx = [np.argmax(bascomp)]
        for k, i in enumerate(idx):
            if k == 0:
                Jposs = basis_descr[i][choice]
            else:
                # take intersection among sets of significant terms or basis states
                Jposs = Jposs.intersection(basis_descr[i][choice])
        Jpossl.append(Jposs)
    df_soE['J_poss'] = Jpossl
    df_soE['nposs'] = [len(s) for s in Jpossl]
    return 

In [41]:
thrpct = 10.
find_possible_J(df_soE, somat['basis'], magpct, 'term', thrpct=thrpct)
# Add term composition vector (complete and also rounded)
df_soE['TC_approx'] = list(np.round(term_compos, 1).T)
df_soE['term_comp'] = list(term_compos.T)
# make column for final J values
df_soE['J'] = None  # not yet assigned

In [42]:
# Check for 0-possibility problems
subdf = df_soE[df_soE.nposs < 1]
n0 = len(subdf)  # number of levels with no possible values of J
# Check for insufficient possibilities
inadequate = []  # list of J with insufficient possible values
for J in J_left.keys():
    dfJ = m2.Jposs_subdf(df_soE, J)
    nposs = len(dfJ)
    if nposs < J_left[J]:
        inadequate.append(J)
        display(dfJ)
if n0 or inadequate:
    if n0:
        print('*** Some levels have all possibilities eliminated ***')
    if inadequate:
        print(f'*** Some J values have insufficient possible levels: {inadequate}')
    print(f'\nThis means that the term weights are defective at the {thrpct}% level.')
    print('If you don\'t care, you can try increasing that threshold ("thrpct").')

In [43]:
# Make backup copies in case J assignment fails at first
df_soE_bak = df_soE.copy()
J_left_bak = J_left.copy()

In [44]:
# To assign J, first try Kmeans clustering based upon energy and term composition
#     as done in atomic_soc8.ipynb

# Assemble vectors needed by Kmeans()
Escale = 10.  # this many cm-1 is on par with 1% composition
Xwt = np.transpose(term_compos)
xe = df_soE.Erel.values / Escale
xe = xe.reshape((-1,1))
X = np.hstack((Xwt, xe))
nlevel = sum(J_all.values())
print(f'Assigning J using {nlevel} clusters/levels...')
Kmean = KMeans(n_clusters=nlevel, n_init=10)
try:
    Kmean.fit(X)
    J_fail = False
except:
    print('*** Clustering failed ***')
    J_fail = True
impossl = []
if not J_fail:
    xmeans = [x[-1] for x in Kmean.cluster_centers_]  # cluster mean (Erel/Escale) values
    Emeans = np.array(xmeans) * Escale
    df_soE['ilev'] = Kmean.labels_
    Jfound = []
    J_fail = False
    for ilev, grp in df_soE.groupby('ilev'):
        g = len(grp)
        J = np.round((g-1)/2, 1)
        df_soE.loc[df_soE.ilev == ilev, 'J'] = J
        Jfound.append(J)
    # Error-checking of assignments
    if Counter(Jfound) != J_all:
        print('*** J values found are not J values expected! ***')
        print('Expected:', sorted(J_all.items()))
        J_clust = Counter(Jfound)
        print('Found   :', sorted(J_clust.items()))
        J_fail = True
    for irow, row in df_soE.iterrows():
        if row.J not in row.J_poss:
            print(f'Level {irow} has J = {row.J} but that is not among ' +
                  f'possibilities {sorted(row.J_poss)}')
            J_fail = True
            impossl.append(irow)
    if len(impossl):
        print('\n*** These "impossible" assignments may mean that term compositions are unreliable ***')
        display(df_soE.loc[impossl, ['Erel', 'J_poss', 'TC_approx', 'J']].style.format(fmt))
if not J_fail:
    print('Success!')

Assigning J using 15 clusters/levels...
Success!


In [45]:
if J_fail:
    print('*** Kmeans failed to assign reasonable values of J ***')
    print('Examine the calculated levels for possible assignments of J')
    display(df_soE_bak[['Erel', 'J_poss', 'nposs', 'TC_approx']])

In [46]:
def J_best_group_for_J(df_soE, J_left, J):
    # For the given value of J, find the tightest groups of levels
    #    that are reasonably isolated
    # Return the groups as list of:
    #    indices, marginal DataFrame, E-spread, TC-spread, E-margin, TC-margin
    # KKI 11/1/2024
    degen = int(2*J + 1)
    ngroup = J_left[J] // degen
    df_unassigned = df_soE[df_soE['J'].isnull()]
    dfJ = m2.Jposs_subdf(df_unassigned, J)
    #display(dfJ[['Erel', 'J_poss', 'TC_approx']])
    ilo = []
    sprE = []
    sprTC = []
    margE = []
    margTC = []
    subdf_list = []
    for i in range(len(dfJ) - degen + 1):
        subdf = dfJ.iloc[i : i+degen]
        Espr, TCspr, maxTC = m2.spreads_ETC(subdf)
        if Espr is None:
            # set to zero
            Espr = maxTC = 0.
            TCspr = np.zeros(len(df_soE.loc[0, 'term_comp']))
        ilo.append(i)
        sprE.append(Espr)
        sprTC.append(maxTC)
        Esep = TCsep = np.inf  # separation from adjacent levels
        llim = i
        ulim = i + degen
        if i > 0:
            llim -= 1
            spr, cspr, maxcspr = m2.spreads_ETC(dfJ.iloc[i-1 : i+degen])
            Esep = min(Esep, spr - Espr)
            TCsep = min(TCsep, maxcspr - maxTC)
        if (i + degen) < len(dfJ):
            ulim += 1
            spr, cspr, maxcspr = m2.spreads_ETC(dfJ.iloc[i : i+degen+1])
            Esep = min(Esep, spr - Espr)
            TCsep = min(TCsep, maxcspr - maxTC)
        margE.append(Esep)
        margTC.append(maxTC)
        subdf_list.append(dfJ.iloc[llim:ulim][['Erel', 'J_poss', 'nposs', 'TC_approx', 'J']])
    retval = []
    for i in np.argsort(sprE):
        # loop over groupings from tightest to loosest
        if margE[i] < (2 * sprE[i]):
            # degenerate with adjacent level
            continue
        # add to the list
        idx = dfJ.index.values[i : i+degen]
        retval.append([idx, subdf_list[i], sprE[i], sprTC[i], margE[i], margTC[i]])
        if len(retval) == ngroup:
            break
    return retval
##

In [47]:
if J_fail:
    # discard any (failed) assignments
    df_soE = df_soE_bak
    J_left = J_left_bak
    
    # try the simplest tricks
    thr_degen = 5 # cm-1
    thr_tcomp = 3 # percent
    df_soE['ilev'] = -1
    
    def highlight_rows(row):
        if row.name in idx:  # Highlight rows with index 0 and 2
            return ['background-color: lightgreen'] * len(row)
        else:
            return [''] * len(row)
    
    nass_singl = m2.singletons_J(df_soE, J_left, thr_degen, thr_tcomp, verbose=True)
    nass_goldi = m2.poss_count_just_right(df_soE, J_left, thr_degen, thr_tcomp, verbose=True)
    print('\n--- Look for best-separated groups of degeneracy (2J + 1) ---')
    for J in sorted(J_left.keys(), reverse=True):
        if J_left[J] < 1:
            # no more assignments to make
            print(f'J = {J} is fully assigned')
            continue
        degen = int(2 * J + 1)
        print(f'--- look for {J_left[J] // degen} groups with J = {J}')
        candi = J_best_group_for_J(df_soE, J_left, J)
        for idx, dfx, sprE, sprTC, margE, margTC in candi:
            styler = dfx.style.apply(highlight_rows, axis=1)
            display(styler)
            ok = input(f'Do you approve assigning this group to J={chem.halves(J)}? ')
            if 'n' not in ok.lower():
                m2.record_J_assignment(df_soE, J_left, idx, J)
                print('OK')
                # try the easy things again, now that changes were made
                nass_singl = m2.singletons_J(df_soE, J_left, thr_degen, thr_tcomp, verbose=True)
                nass_goldi = m2.poss_count_just_right(df_soE, J_left, thr_degen, thr_tcomp,
                                                      verbose=True)
            else:
                print('discarding possible assignment')
    J_fail = False
    for J, unk in J_left.items():
        if unk > 0:
            J_fail = True
            break
    if J_fail:
        print(f'\n*** There are still {len(df_soE["J"].isnull())} unassigned levels ***')
    else:
        print('\n=== All levels have been assigned ===')

In [48]:
if J_fail:
    print('**** J assignments failed, cannot proceed ****')
    for J, nleft in J_left.items():
        if nleft:
            print(f'J = {J} needs {nleft} levels assigned')
    
    def highlight_unassigned(row):
        if row.J is None:
            return ['background-color: yellow'] * len(row)
        else:
            return [''] * len(row)
    
    # Add energy increment column, display, and quit
    erel = df_soE.Erel.values
    df_soE['dE'] = [0] + list(erel[1:] - erel[:-1])
    styler = df_soE[['Erel', 'dE', 'ilev', 'J', 'J_poss', 'TC_approx']].style
    styler = styler.apply(highlight_unassigned, axis=1)
    #styler = styler.apply(lambda x: ["background: yellow" if v is None else "" for v in x], 
    #          subset=pd.IndexSlice[['J']])
    display(styler.format({'Erel': '{:.1f}', 'dE': '{:.1f}'}))

    sys.exit(1)

### After successful J assignment

In [49]:
# Add leading terms to df_soE (all levels from the SO-CI)
Tlead = []
ileadl = []
for tc in df_soE.term_comp:
    ilead = np.argmax(tc)
    Tlead.append(dfciterm.loc[ilead, 'term'])
    ileadl.append(ilead)
df_soE['Lead'] = Tlead
df_soE['ilead'] = ileadl
# Create new DataFrame of aggregated levels
avg = {}
cols = list(df_soE.columns) + ['idx']
for col in cols:
    avg[col] = []
for ilev, grp in df_soE.groupby('ilev'):
    #print('ilev =', ilev)
    #display(grp)
    for col in cols:
        try:
            avg[col].append(grp[col].mean())
        except TypeError:
            # string such as symbol of leading term
            avg[col].append(grp[col].values[0])
        except KeyError:
            # 'idxl' column not yet created
            avg['idx'].append(list(grp.index.values))
dflev = pd.DataFrame({k: avg[k] for k in cols})
# adjust values of TC_approx
for i, row in dflev.iterrows():
    #dflev.at[i, 'TC_approx'] = np.round(row.term_comp, 1) # fails when length=1
    dflev.at[i, 'TC_approx'] = [np.round(x, 1) for x in row.term_comp]
# sort by energy
dflev.sort_values('E', inplace=True, ignore_index=True)
# Create labels from Term + J
Jlbl = []
for trm, J in zip(dflev.Lead, dflev.J):
    Jlbl.append(f'{trm}_{chem.halves(J)}')
dflev['Jlbl'] = Jlbl

In [50]:
# Add information about scatter among supposedly degenerate sublevels
espread = []
tcspread = []
for ilev, row in dflev.iterrows():
    idx = row.idx
    esp = np.ptp(df_soE.loc[idx, 'Erel'])
    espread.append(esp)
    tc = np.vstack(df_soE.loc[idx, 'term_comp'])
    tcptp = np.ptp(tc, axis=0)
    tcspread.append(np.round(tcptp, 1))
dflev['Erel_spread'] = espread
dflev['TC_spread'] = tcspread
print('Check the table below for questionable grouping of levels')
print('"Erel_spread" shows how much the energies differ within a level (cm-1) (small is good)')
print('"TC_spread" shows how much the term compositions differ within a level (%) (small is good)')
print('"idx" shows which magnetic sublevels in "df_soE" compose each level')
# convert 'idx' from np.int64 to regular int, to reduce ugliness in display
dflev.idx = [[int(i) for i in x] for x in dflev.idx]
dflev[['J', 'Erel', 'Erel_spread', 'TC_spread', 'idx']].style.format(fmt)

Check the table below for questionable grouping of levels
"Erel_spread" shows how much the energies differ within a level (cm-1) (small is good)
"TC_spread" shows how much the term compositions differ within a level (%) (small is good)
"idx" shows which magnetic sublevels in "df_soE" compose each level


,J,Erel,Erel_spread,TC_spread,idx
0,0.0,0.0,0.0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],[0]
1,1.0,7221.8,0.0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],"[1, 2, 3]"
2,2.0,10206.2,0.2,[0. 0. 0. 0. 0. 0. 0. 0. 0.],"[4, 5, 6, 7, 8]"
3,2.0,20190.5,0.2,[0. 0. 0. 0. 0. 0. 0. 0. 0.],"[9, 10, 11, 12, 13]"
4,0.0,28602.7,0.0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],[14]
5,1.0,43227.3,0.0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],"[15, 16, 17]"
6,0.0,45109.6,0.0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],[18]
7,1.0,45436.2,0.0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],"[19, 20, 21]"
8,2.0,45707.4,0.1,[0. 0. 0. 0. 0. 0. 0. 0. 0.],"[22, 23, 24, 25, 26]"
9,1.0,55225.2,0.0,[0. 0. 0. 0. 0. 0. 0. 0. 0.],"[27, 28, 29]"


In [51]:
# Convert term compositions 'TC_approx' from numpy arrays to dicts, sorted from largest
#    to smallest weight, and with negligible weights omitted
twl = []
for TC in dflev.TC_approx:
    if TC is None:
        twl.append(None)
    else:
        twd = {dfciterm.at[i,'term']: wt for i, wt in enumerate(TC) if wt >= 0.1}
        tcsort = chem.sort_dict_by_value(twd, reverse=True)
        twl.append(tcsort)
dflev['termwt'] = twl

In [52]:
print('Level assignments from the calculation:')
showcols = ['Lead', 'J', 'Jlbl', 'Erel', 'Eshift', 'termwt']
# convert 'termwt' values from np.float64 to regular float to reduce ugliness upon display
dflev.termwt = [{k: float(v) for k, v in d.items()} for d in dflev.termwt]
display(dflev[showcols].style.format(fmt))

Level assignments from the calculation:


,Lead,J,Jlbl,Erel,Eshift,termwt
0,(1)3P,0.0,(1)3P_0,0.0,-10883.9,"{'(1)3P': 86.6, '(1)1S': 12.7, '(2)1S': 0.4, '(2)3P': 0.3}"
1,(1)3P,1.0,(1)3P_1,7221.8,-3662.1,"{'(1)3P': 99.5, '3S': 0.3, '1P': 0.1, '3D': 0.1}"
2,(1)3P,2.0,(1)3P_2,10206.2,-677.7,"{'(1)3P': 59.6, '(1)1D': 39.8, '3D': 0.5}"
3,(1)1D,2.0,(1)1D_2,20190.5,9306.6,"{'(1)1D': 59.6, '(1)3P': 40.1, '3D': 0.1, '(2)3P': 0.1, '(2)1D': 0.1}"
4,(1)1S,0.0,(1)1S_0,28602.7,17718.8,"{'(1)1S': 87.0, '(1)3P': 12.7, '(2)3P': 0.1, '(2)1S': 0.1}"
5,3D,1.0,3D_1,43227.3,32343.4,"{'3D': 70.9, '1P': 25.9, '3S': 3.1}"
6,(2)3P,0.0,(2)3P_0,45109.6,34225.7,"{'(2)3P': 78.4, '(2)1S': 21.0, '(1)3P': 0.4, '(1)1S': 0.1}"
7,(2)3P,1.0,(2)3P_1,45436.2,34552.3,"{'(2)3P': 49.6, '3S': 24.7, '1P': 14.0, '3D': 11.5, '(1)3P': 0.2}"
8,3D,2.0,3D_2,45707.4,34823.5,"{'3D': 57.1, '(2)1D': 25.4, '(2)3P': 17.1, '(1)1D': 0.2, '(1)3P': 0.1}"
9,1P,1.0,1P_1,55225.2,44341.3,"{'1P': 46.6, '(2)3P': 35.7, '3D': 17.3, '(1)3P': 0.2, '3S': 0.2}"


In [53]:
# Are there duplicated leading terms?
dups = False
unlead = set() # terms that are not leading
for j, grp in dflev.groupby('J'):
    leads = list(grp.Lead)
    if len(leads) > len(set(leads)):
        print(f'*** Duplicate leading term for J = {j} ***')
        dups = True
        for lead in set(leads):
            leads.remove(lead)
        dfdup = grp[grp.Lead.isin(leads)].copy()
        #display(dfdup[showcols].style.format(fmt))
        styler = grp[showcols].style
        styler = styler.apply(lambda x: ["background: yellow" if v in leads else "" for v in x], 
              subset=pd.IndexSlice[['Lead']])
        display(styler.format(fmt))
        for twd in dfdup.termwt:
            for trm, wt in twd.items():
                if trm not in list(grp.Lead):
                    unlead.add(trm)
        print('Terms not leading: ', unlead)

*** Duplicate leading term for J = 2.0 ***


,Lead,J,Jlbl,Erel,Eshift,termwt
2,(1)3P,2.0,(1)3P_2,10206.2,-677.7,"{'(1)3P': 59.6, '(1)1D': 39.8, '3D': 0.5}"
3,(1)1D,2.0,(1)1D_2,20190.5,9306.6,"{'(1)1D': 59.6, '(1)3P': 40.1, '3D': 0.1, '(2)3P': 0.1, '(2)1D': 0.1}"
8,3D,2.0,3D_2,45707.4,34823.5,"{'3D': 57.1, '(2)1D': 25.4, '(2)3P': 17.1, '(1)1D': 0.2, '(1)3P': 0.1}"
10,3D,2.0,3D_2,55868.5,44984.6,"{'3D': 40.4, '(2)3P': 39.7, '(2)1D': 19.6, '(1)1D': 0.3}"
13,(2)1D,2.0,(2)1D_2,58232.1,47348.2,"{'(2)1D': 54.9, '(2)3P': 43.1, '3D': 1.8, '(1)3P': 0.2}"


Terms not leading:  {'(2)3P'}


In [54]:
# Change assignments of any duplicates
if dups:
    print('Correct the duplicate term assignments, if you wish')
    for trm in unlead:
        ifix = input(f'Level to label with {trm} (blank to ignore)? ')
        if len(ifix):
            ifix = int(ifix)
            dflev.loc[ifix, 'Lead'] = trm
            iterm = dfciterm.index[dfciterm.term == trm]
            dflev.loc[ifix, 'ilead'] = iterm
    # rebuild 'Jlbl' values
    jlbl = [f'{t}_{chem.halves(j)}' for t, j in zip(dflev.Lead, dflev.J)]
    dflev['Jlbl'] = chem.enumerative_prefix(jlbl)
    display(dflev[['Lead', 'J', 'Jlbl', 'Erel', 'Eshift']].style.format(fmt))

Correct the duplicate term assignments, if you wish


Level to label with (2)3P (blank to ignore)?  


,Lead,J,Jlbl,Erel,Eshift
0,(1)3P,0.0,(1)3P_0,0.0,-10883.9
1,(1)3P,1.0,(1)3P_1,7221.8,-3662.1
2,(1)3P,2.0,(1)3P_2,10206.2,-677.7
3,(1)1D,2.0,(1)1D_2,20190.5,9306.6
4,(1)1S,0.0,(1)1S_0,28602.7,17718.8
5,3D,1.0,3D_1,43227.3,32343.4
6,(2)3P,0.0,(2)3P_0,45109.6,34225.7
7,(2)3P,1.0,(2)3P_1,45436.2,34552.3
8,3D,2.0,(1)3D_2,45707.4,34823.5
9,1P,1.0,1P_1,55225.2,44341.3


In [55]:
# Check for problems in assignments
nAssign = len(set(dflev.Lead))
nTerm = len(dfciterm)
dropT = False
if nAssign != nTerm:
    print(f'*** I started with {nTerm} terms but have {nAssign} leading terms ***')
    print('Starting: ', sorted(termsIn))
    termsOut = set(dflev.Lead)
    print('Leading : ', sorted(termsOut))
    if nAssign > nTerm:
        addT = termsOut - termsIn
        print('Added terms: ', addT)
    else:
        dropT = termsIn - termsOut
        print('Dropped terms: ', dropT)
        # Add weights from dropped terms and display
        for term in dropT:
            wtcol = []
            for comp in dflev.termwt:
                pct = comp.get(term, 0)
                wtcol.append(pct)
            dflev[term] = wtcol
            fmt[term] = '{:.1f}'
        print('Weights (%) of dropped terms in levels:')
        display(dflev[['Lead', 'J', 'Jlbl', 'Erel', 'Eshift'] + list(dropT)].style.format(fmt))
nlvl = (2 * dflev.J + 1).sum()  # number of sublevels
if nlvl != dimen:
    print(f'*** I started with {nSO} (sub)levels but now have {nlvl} ***')

In [56]:
# Manually assign any dropped terms
if dropT:
    for drt in dropT:
        ia = input(f'Which level do you want to assign to term {drt} (blank to ignore)? ')
        if len(ia) > 0:
            ia = int(ia)
            dflev.loc[ia, 'Lead'] = drt
            iterm = dfciterm.index[dfciterm.term == drt]
            dflev.loc[ia, 'ilead'] = iterm
    # rebuild 'Jlbl' values
    jlbl = [f'{t}_{chem.halves(j)}' for t, j in zip(dflev.Lead, dflev.J)]
    dflev['Jlbl'] = chem.enumerative_prefix(jlbl)
    display(dflev[['Lead', 'J', 'Jlbl', 'Erel', 'Eshift'] + list(dropT)].style.format(fmt))

In [57]:
# Inversion parity of the calculated levels
irreps_ci = set(dfciterm.irrep)
if (PG == 'Ci') and (len(irreps_ci) == 1):
    if 1 in irreps_ci:
        parity = 'even'
    else:
        parity = 'odd'
else:
    # ask user for parity of interest
    parity = input('Please choose "even" or "odd" parity: ')
print(f'Experimental states will be restricted to parity = {parity}')

Experimental states will be restricted to parity = even


### Read experimental energy levels

In [58]:
charge = Qtot - mrci_meta['nelec']  # number of electrons in the last MRCI
labels_ordinated = False  # flag to prevent multiple (1)(1)(1) etc. 
if charge >= 0:
    atstr = atom + '_' + 'I' * (charge + 1)
else:
    # anion
    atstr = atom + '_neg'
fxl = f'{atstr}_exptl_levels.xlsx'
xlpath = os.sep.join([fdir, fxl])
dfexpt = pd.read_excel(xlpath)
if type(dfexpt.J.values[0]) is str:
    fmt['J'] = '{:s}'
print(f'Experimental energy levels read from {fxl}')
# If there is a column "comment", replace NaN with ''
if 'comment' in dfexpt.columns:
    dfexpt['comment'] = dfexpt['comment'].fillna('')

Experimental energy levels read from Pb_I_exptl_levels.xlsx


In [59]:
# Find the number of decimal places in the level energies
Ecol = 'Level (cm-1)'  # the exptl energy column
ndecim = 0
for e in dfexpt[Ecol]:
    words = str(e).split('.')
    # count numeric digits
    n = sum(c.isdigit() for c in words[-1])
    ndecim = max(n, ndecim)
print(f'Experimental energies are provided to {ndecim} decimal digits')
# display formatting
fmt[Ecol] = '{:.' + str(ndecim) + 'f}'

Experimental energies are provided to 4 decimal digits


In [60]:
# Delete any ionization limit
ilim = dfexpt[dfexpt.Term == 'Limit'].index.min()
# delete the "Limit" row and everything past it
n1 = len(dfexpt)
dfexpt = dfexpt.truncate(after=ilim-1)
n2 = len(dfexpt)
if n2 < n1:
    print(f'Discarding {n1-n2} ionized or metastable states')
oddstr = r'\*$|°' # characters to identify terms of odd parity
# Sometimes parity is shown in configuration alone?
#dfeven = dfexpt[~(dfexpt.Term.str.contains(oddstr) | dfexpt.Configuration.str.contains(oddstr))].copy()
#dfodd = dfexpt[dfexpt.Term.str.contains(oddstr) | dfexpt.Configuration.str.contains(oddstr)].copy()
dfeven = dfexpt[~(dfexpt.Term.str.contains(oddstr))].copy()
dfodd = dfexpt[dfexpt.Term.str.contains(oddstr)].copy()
print(f'{len(dfexpt)} experimental levels ({len(dfeven)} even and {len(dfodd)} odd)')
# Select by parity
if parity == 'even':
    # discard odd levels ('Term' field ends with '*')
    dfexpt = dfeven.copy()
elif parity == 'odd':
    dfexpt = dfodd.copy()
else:
    chem.print_err('', f'Parity of "{parity}" is not recognized')
n3 = len(dfexpt)
print(f'{n3} levels accepted for parity = {parity}')
# Reject bad values of J
for i in dfexpt.index:
    try:
        chem.halves_to_float(dfexpt.loc[i, 'J'])
    except ValueError:
        dfexpt.at[i, 'J'] = np.nan
nbad = dfexpt.J.isna().sum()
if nbad:
    print(f'** Rejecting {nbad} levels with malformed J values')
    dfexpt = dfexpt.dropna()
    n4 = len(dfexpt)
    print(f'{n4} level retained')
# Assign unique term symbols
if not labels_ordinated:
    dfexpt = chem.unique_labels_exptl_terms(dfexpt, verbose=True, always=True)
    labels_ordinated = True
# Add column for degeneracy
dfexpt['degen'] = (2 * dfexpt.J.apply(chem.halves_to_float)).astype(int) + 1
# Remove any troublesom non-ascii whitespace characters
dfexpt.Term = [' '.join(t.split()) for t in dfexpt.Term]
Ecm = 'Level (cm-1)'  # label for exptl energy column
dfexpt

Discarding 7 ionized or metastable states
126 experimental levels (58 even and 68 odd)
58 levels accepted for parity = even


,Configuration,Term,J,Level (cm-1),uTerm,degen
0,6s26p2,"(1/2,1/2)",0,0.0000,"(1)(1/2,1/2)",1
1,6s26p2,"(3/2,1/2)",1,7819.2626,"(1)(3/2,1/2)",3
2,6s26p2,"(3/2,1/2)",2,10650.3271,"(1)(3/2,1/2)",5
3,6s26p2,"(3/2,3/2)",2,21457.7982,"(1)(3/2,3/2)",5
4,6s26p2,"(3/2,3/2)",0,29466.8303,"(1)(3/2,3/2)",1
7,6s26p7p,"(1/2,1/2)",1,42918.6434,"(2)(1/2,1/2)",3
8,6s26p7p,"(1/2,1/2)",0,44400.8898,"(2)(1/2,1/2)",1
9,6s26p7p,"(1/2,3/2)",1,44674.9859,"(1)(1/2,3/2)",3
10,6s26p7p,"(1/2,3/2)",2,44809.3636,"(1)(1/2,3/2)",5
19,6s26p8p,"(1/2,1/2)",1,51320.5981,"(3)(1/2,1/2)",3


## Match experimental and theoretical levels

In [61]:
def match_term_symbol(symb_expt, symb_calc):
    # Return True if they are the same, else False
    # Tolerate extra prefix '(1)' or 'a' in symb_expt
    a = str(symb_expt)
    b = str(symb_calc)
    retval = (a == b)  # exact match
    tx = ''
    if ('(1)' in a[:-len(b)]) or ('a' in a):
        tx = a[-len(b):]   # match last characters
        retval |= (b == tx)
    return retval

In [62]:
def termwt_to_dict(termwt):
    # Given a list/Series of term weight arrays,
    # Return a corresponding list of dict where key = term label
    twl = []
    for TC in termwt:
        if TC is None:
            twl.append(None)
        else:
            twd = {dfciterm.at[i,'term']: wt for i, wt in enumerate(TC) if wt >= 0.1}
            # sort from largest to smallest weights
            twd = chem.sort_dict_by_value(twd, reverse=True)
            twl.append(twd)
    return twl

In [63]:
def match_theory_to_expt(dfth, dfx, bigerr=3000):
    '''
    Match experimental levels to theoretical
    Return a DataFrame containing both theory and expt, and
      the index of the highest level matched
    'bigerr' is in cm-1 and triggers extra scrutiny
    '''
    print('Matching experimental levels with theoretical levels')
    Jlist = sorted(set(dfth.J))
    dfcomp = dfexpt.copy()
    dfcomp['Tcalc'] = ''  # term assignment in computation
    dfcomp['leadwt'] = ''
    dfcomp['Ecalc'] = np.nan
    dfcomp['termwt'] = None
    dfcomp['Composition'] = None
    imax = 0  # index of highest exptl level matched
    for J in Jlist:
        print(f'J = {J}')
        # get the indices of the exptl levels that best match theoretical
        dfJth = dfth[dfth.J == J]
        #display(dfJth[['Lead', 'J', 'Erel', 'TC_approx']])
        dfJx = dfx[dfx.J == J]
        if len(dfJx) < 1:
            # try str representation
            hJstr = chem.halves(J)  # as str fraction
            dfJx = dfx[dfx.J == hJstr]
        #display(dfJx)
        idx = match_th_expt_1J_V2(dfJth, dfJx, bigerr=bigerr)
        #print(f'>>>J = {J}, num theor = {len(dfJth)}, idx = {idx}')
        for i, ix in enumerate(idx):
            rowth = dfJth.iloc[i]
            dfcomp.at[ix, 'Tcalc'] = rowth.Lead
            dfcomp.at[ix, 'leadwt'] = rowth.TC_approx[term_index(rowth.Lead)]
            dfcomp.at[ix, 'Ecalc'] = rowth.Erel
            dfcomp.at[ix, 'termwt'] = rowth.TC_approx
            dfcomp.at[ix, 'Composition'] = [x for x in rowth.term_comp] # fix when length=1
            imax = max(imax, ix)
    # Convert approx. 'termwt' from numpy arrays to dicts, sorted from largest to 
    #    smallest weight, and with negligible weights omitted
    dfcomp['termwt'] = termwt_to_dict(dfcomp.termwt)
    return dfcomp, imax

In [64]:
def match_th_expt_1J_V2(dfJth, dfJx, bigerr=3000):
    '''
    Given DataFrame's of theoretical and experimental levels for the same J,
    Return a list of indices into dfJx that match those in dfJth
        based upon energy, but with preference for same leading term
    'bigerr' is in cm-1 and triggers extra scrutiny
    '''
    idx = []  # index of exptl level that matches each theor level
    for i, row in dfJth.iterrows():
        Eth = row.Erel
        thTerm = row.Lead

        def same_Lead(xTerm):
            # Does the exptl "uTerm" match the theoretical "Lead"?
            return match_term_symbol(xTerm, thTerm)
    
        # Is there an exptl level with the same leading term?
        termatch = dfJx.uTerm.apply(same_Lead)
        subx = dfJx.loc[termatch]
        if len(subx) == 1:
            # The nicest situation, but check the energy
            err = abs(Eth - subx[Ecol].values[0])
            if err < bigerr:
                # tolerable
                print(f'    exptl term "{subx.Term.iloc[0]}" matches leading "{thTerm}"')
                j = subx.index.values[0]
                idx.append(j)
                continue
            else:
                pass
                #print(f'    exptl term "{subx.Term.iloc[0]}" rejected as match for' +
                #      f' leading "{thTerm}" because energy error = {err:.0f} cm-1')
        # There are multiple or no matching terms
        #   rely upon energy ordering (among matching terms, if any)
        if len(subx) > 1:
            # Exptl data assign the same leading term to multiple levels of this J (unusual)
            #print(f'    leading "{thTerm}" matches multiple exptl terms "{subx.Term.values}"')
            pass
        else:
            # No matching term symbols
            #print(f'    leading "{thTerm}" matches no exptl term labels')
            subx = dfJx.sort_values(Ecm)  # consider all levels because none match the term symbol
        j = None
        for j, rowx in subx.iterrows():
            # expect energy ordering to be similar
            #    there is no way to detect inversions
            if j in idx:
                # this exptl level already matched
                continue
            # we get here if this level has not yet been matched; take the first one
            print(f'    leading "{thTerm}" matched with exptl "{rowx.Term}"')
            # check the energy
            err = Eth - rowx[Ecol]
            if abs(err) > bigerr:
                print(f'\t*** but large energy error of {err:.0f} cm-1 ***')
                if err < 0:
                    print('\t*** maybe an experimental level is missing')
            break
        if (j is not None) and (j not in idx):
            idx.append(j)
    return idx

In [65]:
# Match theoretical and experimental levels
# imax is the index of the highest-energy level matched
dfdiff, imax = match_theory_to_expt(dflev, dfexpt)
dfdiff['err'] = dfdiff.Ecalc - dfdiff[Ecol]

Matching experimental levels with theoretical levels
J = 0.0
    leading "(1)3P" matched with exptl "(1/2,1/2)"
    leading "(1)1S" matched with exptl "(3/2,3/2)"
    leading "(2)3P" matched with exptl "(1/2,1/2)"
    leading "(2)1S" matched with exptl "(1/2,1/2)"
	*** but large energy error of 9264 cm-1 ***
J = 1.0
    leading "(1)3P" matched with exptl "(3/2,1/2)"
    leading "3D" matched with exptl "(1/2,1/2)"
    leading "(2)3P" matched with exptl "(1/2,3/2)"
    leading "1P" matched with exptl "(1/2,1/2)"
	*** but large energy error of 3905 cm-1 ***
    leading "3S" matched with exptl "(1/2,3/2)"
	*** but large energy error of 4950 cm-1 ***
J = 2.0
    leading "(1)3P" matched with exptl "(3/2,1/2)"
    leading "(1)1D" matched with exptl "(3/2,3/2)"
    leading "3D" matched with exptl "(1/2,3/2)"
    leading "3D" matched with exptl "(1/2,3/2)"
	*** but large energy error of 3924 cm-1 ***
    leading "(2)1D" matched with exptl "2[5/2]"
	*** but large energy error of 5382 cm-1 ***
J 

In [66]:
t = dfdiff.loc[imax, 'Tcalc']
J = dfdiff.loc[imax, 'J']
levmax = f'{t}_{J}'
emax = dfdiff.loc[imax, Ecol]
showcols = ['Configuration', 'uTerm', 'Tcalc', 'J', Ecol, 'Ecalc', 'err', 'termwt']
print(f'Highest level matched is {levmax} at {emax} cm-1 (exptl energy)')
# Notify about any exptl levels that the calculation skipped over
dfskipped = dfdiff[dfdiff.Ecalc.isna() & (dfdiff[Ecol] < emax)]
if len(dfskipped):
    print('\n*** The following experimental levels are skipped over in the calculation ***')
    print(  '***     Consider doing another calculation with these levels included     ***')
    display(dfskipped[showcols])
    #display(dfdiff.loc[:imax][showcols])

Highest level matched is (2)1D_2 at 52849.7991 cm-1 (exptl energy)


In [67]:
# Convert str values of J to float
dfdiff['J'] = dfdiff.J.apply(chem.halves_to_float)
fmt['J'] = '{:.1f}'
warnThresh = 1000  # highlight errors larger than this (cm-1)
# drop rows with NaN (no matching level in the calculation)
dfdiff_all = dfdiff.copy()
dfdiff.dropna(axis=0, inplace=True)
# sort by increasing calculated energy, and re-index
dfdiff.sort_values('Ecalc', inplace=True, ignore_index=True)
# Update the exptl term enumerative prefixes to reflect this subset of terms
dfdiff['uTerm'] = chem.update_enumerative_prefix(dfdiff.uTerm)
selcols = ['Configuration', 'uTerm', 'J', Ecol, 'Tcalc', 'Ecalc', 'err']
# Print a warning if experimental levels are missing
nth = len(dflev); ndiff = len(dfdiff)
expt_missing = nth - ndiff
if expt_missing > 0:
    print(f'\n**** There are {nth} theoretical levels but only {ndiff} ' +
          'matching experimental levels ****\n')
    setdiff = set(dfdiff.Ecalc)
    setth = set(dflev.Erel)
    missing = setth - setdiff
    dfunmatched = dflev[dflev.Erel.isin(missing)].copy()
    print('Unmatched theoretical levels:')
    display(dfunmatched[['Erel', 'J', 'Lead', 'Jlbl', 'termwt']].style.format(fmt))
else:
    # use as flag
    expt_missing = 0

*** Warning: "Failed to parse label "(1)(1/2,1/2)"" in strip_enumerative_prefix()
*** Warning: "Failed to parse label "(1)(3/2,1/2)"" in strip_enumerative_prefix()
*** Warning: "Failed to parse label "(1)(3/2,3/2)"" in strip_enumerative_prefix()
*** Warning: "Failed to parse label "(2)(1/2,1/2)"" in strip_enumerative_prefix()
*** Warning: "Failed to parse label "(1)(1/2,3/2)"" in strip_enumerative_prefix()
*** Warning: "Failed to parse label "(3)(1/2,1/2)"" in strip_enumerative_prefix()
*** Warning: "Failed to parse label "(2)(1/2,3/2)"" in strip_enumerative_prefix()


In [ ]:
# change to normal datatypes to make display less ugly
dfdiff['termwt'] = [{k: float(v) for k, v in d.items()} for d in dfdiff.termwt]
dfdiff['Composition'] = [[float(c) for c in lis] for lis in dfdiff.Composition]
#dfdiff.style.format(fmt)

In [68]:
if expt_missing:
    print('*** Installing theoretical levels where no exptl levels could be matched ***')
    dfu = dfunmatched.rename(columns={'Lead': 'Tcalc', 'Erel': 'Ecalc', 
                'term_comp': 'Composition'})
    dfu['degen'] = (2 * dfu.J + 1).astype(int)
    # add column for weight of leading term
    lwt = [twt[lt] for lt, twt in zip(dfunmatched.Lead, dfunmatched.termwt)]
    dfu['leadwt'] = lwt
    # install theoretical energies as "experimental"
    dfu[Ecol] = dfu.Ecalc
    dfu['err'] = 0
    dfu['Configuration'] = ''
    dfu['Term'] = ''
    dfu['uTerm'] = ''
    # drop unwanted columns
    dfu = dfu.drop(columns=['Eshift', 'J_poss', 'nposs', 'TC_approx', 'ilev', 'ilead',
                     'idx', 'Jlbl', 'Erel_spread', 'TC_spread', 'E'])
    dfdiff = pd.concat([dfdiff, dfu])
    dfdiff = dfdiff.sort_values(Ecol).reset_index(drop=True)

In [69]:
if 'comment' in dfdiff.columns:
    selcols.append('comment')
print(f'Please inspect the following pairing of theory ("Ecalc") with expt ("{Ecol}")')
print(f'Errors > {warnThresh} cm-1 are highlighted in yellow')
show_skipped = False  # display the skipped levels (if any) in the table below
if len(dfskipped) and show_skipped:
    # also show the skipped/unmatched exptl levels
    styler = dfdiff_all[showcols].style
else:
    styler = dfdiff[showcols].style
styler = styler.apply(lambda x: ["background: yellow" if abs(v) > warnThresh else "" for v in x], 
              subset=pd.IndexSlice[['err']])
if (len(dfskipped) == 0) or (not show_skipped):
    print('Disagreements in term assignments are highlighted in red')
    styler = styler.apply(lambda x: (x != dfdiff['uTerm']).map({True: "background-color: red; \
                  color: white", False: ""}), subset=['Tcalc'])
display(styler.format(fmt))

Please inspect the following pairing of theory ("Ecalc") with expt ("Level (cm-1)")
Errors > 1000 cm-1 are highlighted in yellow
Disagreements in term assignments are highlighted in red


,Configuration,uTerm,Tcalc,J,Level (cm-1),Ecalc,err,termwt
0,6s26p2,"(1)(1/2,1/2)",(1)3P,0.0,0.0000,0.0,0.0,"{'(1)3P': np.float64(86.6), '(1)1S': np.float64(12.7), '(2)1S': np.float64(0.4), '(2)3P': np.float64(0.3)}"
1,6s26p2,"(1)(3/2,1/2)",(1)3P,1.0,7819.2626,7221.8,-597.5,"{'(1)3P': np.float64(99.5), '3S': np.float64(0.3), '1P': np.float64(0.1), '3D': np.float64(0.1)}"
2,6s26p2,"(1)(3/2,1/2)",(1)3P,2.0,10650.3271,10206.2,-444.1,"{'(1)3P': np.float64(59.6), '(1)1D': np.float64(39.8), '3D': np.float64(0.5)}"
3,6s26p2,"(1)(3/2,3/2)",(1)1D,2.0,21457.7982,20190.5,-1267.3,"{'(1)1D': np.float64(59.6), '(1)3P': np.float64(40.1), '3D': np.float64(0.1), '(2)3P': np.float64(0.1), '(2)1D': np.float64(0.1)}"
4,6s26p2,"(1)(3/2,3/2)",(1)1S,0.0,29466.8303,28602.7,-864.2,"{'(1)1S': np.float64(87.0), '(1)3P': np.float64(12.7), '(2)3P': np.float64(0.1), '(2)1S': np.float64(0.1)}"
5,6s26p7p,"(2)(1/2,1/2)",3D,1.0,42918.6434,43227.3,308.6,"{'3D': np.float64(70.9), '1P': np.float64(25.9), '3S': np.float64(3.1)}"
6,6s26p7p,"(2)(1/2,1/2)",(2)3P,0.0,44400.8898,45109.6,708.7,"{'(2)3P': np.float64(78.4), '(2)1S': np.float64(21.0), '(1)3P': np.float64(0.4), '(1)1S': np.float64(0.1)}"
7,6s26p7p,"(1)(1/2,3/2)",(2)3P,1.0,44674.9859,45436.2,761.2,"{'(2)3P': np.float64(49.6), '3S': np.float64(24.7), '1P': np.float64(14.0), '3D': np.float64(11.5), '(1)3P': np.float64(0.2)}"
8,6s26p7p,"(1)(1/2,3/2)",3D,2.0,44809.3636,45707.4,898.1,"{'3D': np.float64(57.1), '(2)1D': np.float64(25.4), '(2)3P': np.float64(17.1), '(1)1D': np.float64(0.2), '(1)3P': np.float64(0.1)}"
9,6s26p8p,"(3)(1/2,1/2)",1P,1.0,51320.5981,55225.2,3904.6,"{'1P': np.float64(46.6), '(2)3P': np.float64(35.7), '3D': np.float64(17.3), '(1)3P': np.float64(0.2), '3S': np.float64(0.2)}"


## Compute spin-orbit corrections, <em>E</em><sub>so</sub>

In [70]:
# No theoretical calculations are needed to use eq. (1)
# I.e., accept experimental term assignments and assume that each level
#    derives from a single term (100% term weight)
xterms = []  # list of exptl term labels
thterms = [] # list of theoretical term labels
eterms = []  # list of term energies
for term in dfdiff.uTerm:
    if term not in xterms:
        xterms.append(term)
for term in dfdiff.Tcalc:
    if term not in thterms:
        thterms.append(term)
for Term in xterms:
    subdf = dfdiff[dfdiff.uTerm == Term]
    emean = np.dot(subdf.degen, subdf[Ecol]) / subdf.degen.sum()
    eterms.append(emean)
dfeq1 = pd.DataFrame({'Term': xterms, 'Eterm': eterms}).sort_values('Eterm').reset_index(drop=True)
print('In the naive model of eq. (1), experimental term assignments are accepted')
print('    and each level is assumed to derive from a single term (100% term weight).')
print('Term energies (cm-1) using eq. (1) [no theoretical input]:')
display(dfeq1.style.format(fmt))
SOC1 = -1 * np.round(dfeq1.at[0, 'Eterm'], 3)
lowterm = dfeq1.at[0, 'Term']
print(f'The exptl term of lowest energy is *** {lowterm} *** with SOC1 = {SOC1} cm-1')
levterm = dfdiff.uTerm.values[0]

target = levterm

if lowterm != levterm:
    # The lowest term is not the leading term of the lowest level
    SOC1alt = SOC1
    SOC1 = -1 * np.round(dfeq1[dfeq1.Term == levterm]['Eterm'].values[0], 3)
    print(f'The lowest level belongs to \t{levterm} \twith SOC1 = {SOC1} cm-1')

# Check for rare situation
th_term = dfdiff[dfdiff.uTerm == target].Tcalc.values[0]
if th_term != target:
    # hopefully just the enumerative prefix
    print(f'    *** changing target term from {target} to {th_term}')
    old_target = target
    target = th_term
else:
    old_target = None
print()
print(f'Term {target} is selected for calculating the spin-orbit correction')
print('    to change this, assign the variable "target" to another term')

In the naive model of eq. (1), experimental term assignments are accepted
    and each level is assumed to derive from a single term (100% term weight).
Term energies (cm-1) using eq. (1) [no theoretical input]:


,Term,Eterm
0,"(1)(1/2,1/2)",0.0
1,"(1)(3/2,1/2)",9588.7
2,"(1)(3/2,3/2)",22792.6
3,"(2)(1/2,1/2)",43289.2
4,"(1)(1/2,3/2)",44759.0
5,"(3)(1/2,1/2)",51437.0
6,"(2)(1/2,3/2)",51933.9
7,2[5/2],52845.1


The exptl term of lowest energy is *** (1)(1/2,1/2) *** with SOC1 = -0.0 cm-1
    *** changing target term from (1)(1/2,1/2) to (1)3P

Term (1)3P is selected for calculating the spin-orbit correction
    to change this, assign the variable "target" to another term


In [71]:
def term_energy_from_levels(df, term, returnDF=False):
    # Given a DataFrame with columns ['J', 'Composition', Ecol],
    #   where (variable) Ecol is the header for the column of exptl level energies,
    # Return the term's average energy as derived from the levels [eq. (2) in pub.]
    # If 'returnDF', also return a DataFrame for the selected term
    # 'term' is from theory
    iterm = term_index(term)
    if iterm is None:
        # invalid term
        return None, None
    termwt = []  # weight of term "term" in each level
    for tc in df.Composition.values:
        termwt.append(tc[iterm])
    termwt = np.array(termwt)
    dweight = df.degen.values * termwt  # weights should include degeneracies
    # take product of experimental energies and theoretical term weights
    Eterm = np.dot(df[Ecol], dweight) / dweight.sum()
    if not returnDF:
        return Eterm
    # Construct DF showing distribution of term among levels
    cols = ['Configuration', 'uTerm', 'Tcalc', 'J', 'degen', Ecol, 'Ecalc', 'err']
    df_distrib = df[cols].copy()
    df_distrib.insert(0, 'weight', termwt / 100)  # fraction instead of percent
    return Eterm, df_distrib

In [72]:
print(f'Term of interest is {target}')

Term of interest is (1)3P


In [73]:
print(f'Distribution of target term {target} among levels:')
Eterm, df_distrib = term_energy_from_levels(dfdiff, target, returnDF=True)
# Combine degeneracy with weight
df_distrib.insert(0, target, df_distrib.weight * df_distrib.degen)
subdf = df_distrib[df_distrib[target] > 1.e-6].copy()
# suppress redundant columns
subdf = subdf.drop(columns=['weight', 'degen'])
styler = subdf.sort_values(target, ascending=False).style
styler = styler.apply(lambda x: ["background: yellow" if abs(v) > warnThresh else "" for v in x], 
              subset=pd.IndexSlice[['err']])
display(styler.format(fmt))
ibad = subdf[subdf.err.abs() > 3000]
print(f'Sum of displayed weights = {subdf[target].sum():.5f}')

Distribution of target term (1)3P among levels:


,(1)3P,Configuration,uTerm,Tcalc,J,Level (cm-1),Ecalc,err
1,2.986247,6s26p2,"(1)(3/2,1/2)",(1)3P,1.0,7819.2626,7221.8,-597.5
2,2.980399,6s26p2,"(1)(3/2,1/2)",(1)3P,2.0,10650.3271,10206.2,-444.1
3,2.005509,6s26p2,"(1)(3/2,3/2)",(1)1D,2.0,21457.7982,20190.5,-1267.3
0,0.866034,6s26p2,"(1)(1/2,1/2)",(1)3P,0.0,0.0000,0.0,0.0
4,0.127384,6s26p2,"(1)(3/2,3/2)",(1)1S,0.0,29466.8303,28602.7,-864.2
13,0.009256,6s26p(2P°1/2)5f,2[5/2],(2)1D,2.0,52849.7991,58232.1,5382.3
9,0.005799,6s26p8p,"(3)(1/2,1/2)",1P,1.0,51320.5981,55225.2,3904.6
7,0.004531,6s26p7p,"(1)(1/2,3/2)",(2)3P,1.0,44674.9859,45436.2,761.2
6,0.004478,6s26p7p,"(2)(1/2,1/2)",(2)3P,0.0,44400.8898,45109.6,708.7
12,0.003420,6s26p8p,"(2)(1/2,3/2)",3S,1.0,51916.9408,56867.1,4950.1


Sum of displayed weights = 9.00000


In [74]:
if False:
    # make DataFrame of ground term for export to Excel 
    print(fname)
    dfexport = subdf[[target, 'J', Ecol, 'Ecalc', 'err']].rename(columns={Ecol: 'Expt/cm-1'})
    dfexport['termwts'] = dflev.termwt
    # round theoretical energies to nearest 0.1 cm-1
    for col in ['Ecalc', 'err']:
        dfexport[col] = np.round(dfexport[col], 1)
    display(dfexport)
    dfexport.to_clipboard(index=False)
    print('-'*20, 'Table copied to clipboard', '-'*20)

pb_3PDPS1DSPDS_actzpp.pro


,(1)3P,J,Expt/cm-1,Ecalc,err,termwts
0,0.866034,0.0,0.0000,0.0,0.0,"{'(1)3P': 86.6, '(1)1S': 12.7, '(2)1S': 0.4, '..."
1,2.986247,1.0,7819.2626,7221.8,-597.5,"{'(1)3P': 99.5, '3S': 0.3, '1P': 0.1, '3D': 0.1}"
2,2.980399,2.0,10650.3271,10206.2,-444.1,"{'(1)3P': 59.6, '(1)1D': 39.8, '3D': 0.5}"
3,2.005509,2.0,21457.7982,20190.5,-1267.3,"{'(1)1D': 59.6, '(1)3P': 40.1, '3D': 0.1, '(2)..."
4,0.127384,0.0,29466.8303,28602.7,-864.2,"{'(1)1S': 87.0, '(1)3P': 12.7, '(2)3P': 0.1, '..."
5,0.000003,1.0,42918.6434,43227.3,308.6,"{'3D': 70.9, '1P': 25.9, '3S': 3.1}"
6,0.004478,0.0,44400.8898,45109.6,708.7,"{'(2)3P': 78.4, '(2)1S': 21.0, '(1)3P': 0.4, '..."
7,0.004531,1.0,44674.9859,45436.2,761.2,"{'(2)3P': 49.6, '3S': 24.7, '1P': 14.0, '3D': ..."
8,0.002523,2.0,44809.3636,45707.4,898.1,"{'3D': 57.1, '(2)1D': 25.4, '(2)3P': 17.1, '(1..."
9,0.005799,1.0,51320.5981,55225.2,3904.6,"{'1P': 46.6, '(2)3P': 35.7, '3D': 17.3, '(1)3P..."


-------------------- Table copied to clipboard --------------------


In [125]:
if (atom == 'Pb') and (charge == 0):
    # Prepare a table with all three valence terms and write to an Excel file
    print('Special table with all three valence terms of Pb(I)')
    fxl = fname.replace('.pro', '.xlsx')
    print(f'data from {fname}')
    vterms = dfciterm.term.values[:3].tolist()
    emean = []  # the calculated term energies
    for vterm in vterms:
        print(f'Term {vterm}')
        Et, df_d = term_energy_from_levels(dfdiff, vterm, returnDF=True)
        if vterm == target:
            df_dist = df_d.copy()
        # Combine degeneracy with weight
        df_dist[vterm] = df_d.weight * df_d.degen
        print(f'   sum of weights = {df_dist[vterm].sum():.5f}')
        e = np.dot(df_dist[vterm], df_dist[Ecol]) / df_dist[vterm].sum()
        emean.append(np.round(e, 2))
    # delete unimportant rows
    df_dist = df_dist.loc[~(df_dist[vterms] < 1.e-6).all(axis=1)]
    # delete redundant columns
    subdf = df_dist.drop(columns=['weight', 'degen']).sort_values(Ecol, ascending=True)
    # add row for term energies
    lastrow = ['Term energies', '(in cm-1)', ''] + [0]*4 + emean
    subdf.loc[999] = lastrow
    styler = subdf.style
    styler = styler.apply(lambda x: ["background: yellow" if abs(v) > warnThresh else "" for v in x], 
                  subset=pd.IndexSlice[['err']])
    display(styler.format(fmt))
    subdf.to_excel(fxl, index=False)
    print('-'*20, f'Table saved to file {fxl}', '-'*20)

Special table with all three valence terms of Pb(I)
data from pb_3PDPS1DSPDS_actzpp.pro
Term (1)3P
   sum of weights = 9.00000
Term (1)1D
   sum of weights = 5.00000
Term (1)1S
   sum of weights = 1.00000


,Configuration,uTerm,Tcalc,J,Level (cm-1),Ecalc,err,(1)3P,(1)1D,(1)1S
0,6s26p2,"(1)(1/2,1/2)",(1)3P,0.0,0.0000,0.0,0.0,0.866034,0.000000,0.127167
1,6s26p2,"(1)(3/2,1/2)",(1)3P,1.0,7819.2626,7221.8,-597.5,2.986247,0.000000,0.000000
2,6s26p2,"(1)(3/2,1/2)",(1)3P,2.0,10650.3271,10206.2,-444.1,2.980399,1.991038,0.000000
3,6s26p2,"(1)(3/2,3/2)",(1)1D,2.0,21457.7982,20190.5,-1267.3,2.005509,2.980247,0.000000
4,6s26p2,"(1)(3/2,3/2)",(1)1S,0.0,29466.8303,28602.7,-864.2,0.127384,0.000000,0.870282
5,6s26p7p,"(2)(1/2,1/2)",3D,1.0,42918.6434,43227.3,308.6,0.000003,0.000000,0.000000
6,6s26p7p,"(2)(1/2,1/2)",(2)3P,0.0,44400.8898,45109.6,708.7,0.004478,0.000000,0.001495
7,6s26p7p,"(1)(1/2,3/2)",(2)3P,1.0,44674.9859,45436.2,761.2,0.004531,0.000000,0.000000
8,6s26p7p,"(1)(1/2,3/2)",3D,2.0,44809.3636,45707.4,898.1,0.002523,0.012047,0.000000
9,6s26p8p,"(3)(1/2,1/2)",1P,1.0,51320.5981,55225.2,3904.6,0.005799,0.000000,0.000000


-------------------- Table saved to file pb_3PDPS1DSPDS_actzpp.xlsx --------------------


In [ ]:
1/0

In [ ]:
# In case of very large errors, allow replacement of exptl energy with theoretical
ibad = subdf[subdf.err.abs() > 3000]
if len(ibad):
    print('Some errors in level energies are very large:')
    display(ibad.style.format(fmt))
    idxl = []
    while True:
        idx = input('Level to replace exptl energy with theoretical (blank to end): ')
        if idx:
            idxl.append(int(idx))
        else:
            break
    for i in idxl:
        dfdiff.at[i, Ecol] = dfdiff.at[i, 'Ecalc']
        dfdiff.at[i, 'err'] = 0
    if len(idxl):
        print('Energies replaced')
        ibad = ibad.index.values
        display(dfdiff.loc[ibad].style.format(fmt))

In [ ]:
# In case of large errors of opposite sign, consider different match with expt
ibad = subdf[subdf.err.abs() > 3000]
swapped = False
if len(ibad):
    # Is max ~ -min?
    if abs(ibad.err.max() + ibad.err.min()) < 1000:
        imax = ibad.err.idxmax()
        imin = ibad.err.idxmin()
        if ibad.at[imin, 'J'] == ibad.at[imax, 'J']:
            # can only switch equal J
            q = input(f'Swap the matches for rows {imin} and {imax}? ')
            if 'n' == q.lower()[0]:
                print('Keeping old match')
            else:
                print('Swapping matches')
            for col in ['Tcalc', 'Ecalc', 'leadwt', 'termwt', 'Composition']:
                x = dfdiff.at[imin, col]
                dfdiff.at[imin, col] = dfdiff.at[imax, col]
                dfdiff.at[imax, col] = x
            for i in [imin, imax]:
                dfdiff.at[i, 'err'] = dfdiff.at[i, 'Ecalc'] - dfdiff.at[i, Ecol]
            swapped = True

In [ ]:
# Use experimental level energies via eq. (2) (from the publication)
Eterm, df_distrib = term_energy_from_levels(dfdiff, target, returnDF=True)
if swapped:
    # display the modified matching scheme 
    subdf = df_distrib[df_distrib.weight > 1.e-6]
    styler = subdf.sort_values('weight', ascending=False).style
    styler = styler.apply(lambda x: ["background: yellow" if abs(v) > warnThresh else "" for v in x], 
                  subset=pd.IndexSlice[['err']])
    print('Matching scheme after swapping:')
    display(styler.format(fmt))
SOC2 = -Eterm
print('Applying eq. (2) (experimental energies and theoretical term weights).')
print(f'For term {target}, SOC2 = {SOC2:.2f} cm-1')


In [ ]:
print(f'Molpro source file: {fname}\n')
print(f'Alternative values for E_so[{target}] of atom {atom}:')
print('-' * 25)
if old_target is None:
    print('{:12s} {:.2f} cm-1'.format('eq (1)', SOC1))
else:
    print('{:12s} {:.2f} cm-1\t (term {:s})'.format('eq (1)', SOC1, old_target))
print('{:12s} {:.2f} cm-1'.format('raw theory', SOCraw))
print('{:12s} {:.2f} cm-1'.format('eq (2)', SOC2))
print('-' * 25)
print(f'\nDifference [eq. (2)] - [eq. (1)]    = {SOC2-SOC1:.1f} cm-1')
print(f'Difference [eq. (2)] - [raw theory] = {SOC2-SOCraw:.1f} cm-1')

# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [ ]:
# Term energy errors as inferred from all levels
termlist = []
wmean = []
wstds = []
# also consider unsigned (absolute value) errors
uwmean = []
uwstds = []
# also consider RMSE
trmse = []
trmset = set(dfdiff.Tcalc)
trmset.update(unlead)
for t in unlead:
    if t not in thterms:
        thterms.append(t)
for term in trmset:
    eterm, df_distro = term_energy_from_levels(dfdiff, term, returnDF=True)
    if eterm is None:
        print(f'No theoretical term matches exptl {term}')
        continue
    termlist.append(term)
    wts = df_distro.weight.values * df_distro.degen
    m, s = chem.weighted_mean(df_distro.err, wts)
    wmean.append(m)
    wstds.append(s)
    uerr = np.abs(df_distro.err.values)
    um, us = chem.weighted_mean(uerr, wts)
    uwmean.append(um)
    uwstds.append(us)
    umsq, ussq = chem.weighted_mean(uerr ** 2, wts)
    trmse.append(np.sqrt(umsq))
dftermerr = pd.DataFrame({'Term': termlist, 'wmean': wmean, 'wstds': wstds,
                         'uwmean': uwmean, 'uwstds': uwstds, 'rwmse': trmse})
print('Errors in term energies (cm-1) as inferred from the full distribution')
print('    of each term over all levels')
# sort
print(thterms)
dftermerr.Term = pd.Categorical(dftermerr.Term, thterms)
dftermerr = dftermerr.sort_values('Term')
styler = dftermerr.style
styler = styler.apply(lambda x: ["background: yellow" if abs(v) > warnThresh else "" for v in x], 
              subset=pd.IndexSlice[['wmean']])
styler.format(fmt)

### Optional distribution of some other term

In [ ]:
other_term = '1D' #'4P°'
Eterm, df_distrib = term_energy_from_levels(dfdiff, other_term, returnDF=True)
if Eterm is None:
    print(f'Cannot evaluate errors for levels within term "{other_term}"')
else:
    print(f'Distribution of term {other_term} among levels:')
    subdf = df_distrib[df_distrib.weight >= 1.e-6]
    display(subdf.sort_values('weight', ascending=False).style.format(fmt))
    wtot = df_distrib.weight.sum()
    dwtot = np.dot(df_distrib.weight, df_distrib.degen)
    print('Total weight = {:.3f} ({:.3f} including degeneracies)'.format(wtot, dwtot))
    print(f'Eq. (2) term energy = {Eterm:.1f} cm-1')